##### Copyright 2023 Google LLC

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

### Install & import

In [1]:
!pip install -U -q google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.8/160.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 760.0/760.0 kB 36.5 MB/s eta 0:00:00


In [2]:
# Install the client library and import necessary modules.
import google.generativeai as genai

import base64
import copy
import hashlib
import io
import json
import mimetypes
import pathlib
import pprint
import requests


import PIL.Image
import IPython.display
from IPython.display import Markdown

### Mount Google Drive

In [3]:
from google.colab import drive

drive.mount("/gdrive")

Mounted at /gdrive


## Set the API key

Add your API_KEY to the secrets manager in the left panel "🔑".

In [7]:
from google.colab import userdata

GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")

In [8]:
# Configure the client library by providing your API key.
genai.configure(api_key=GOOGLE_API_KEY)

## Parse the arguments

In [9]:
model = 'gemini-1.5-flash' # @param {isTemplate: true}
contents_b64 = 'W10=' # @param {isTemplate: true}
generation_config_b64 = 'eyJ0ZW1wZXJhdHVyZSI6MSwidG9wX3AiOjAuOTUsInRvcF9rIjo2NCwibWF4X291dHB1dF90b2tlbnMiOjgxOTJ9' # @param {isTemplate: true}
safety_settings_b64 = "e30="  # @param {isTemplate: true}

gais_contents = json.loads(base64.b64decode(contents_b64))

generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))

stream = False

In [10]:
gais_contents

[]

In [11]:
generation_config

{'temperature': 1, 'top_p': 0.95, 'top_k': 64, 'max_output_tokens': 8192}

In [12]:
safety_settings

{}

## [optional] Show the conversation

This section displays the conversation received from Google AI Studio.

In [13]:
# @title `show_file` function
def show_file(file_data):
    mime_type = file_data["mime_type"]

    if drive_id := file_data.get("drive_id", None):
        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        name = path
        # data = path.read_bytes()
        kwargs = {"filename": path}
    elif url := file_data.get("url", None):
        name = url
        kwargs = {"url": url}
        # response = requests.get(url)
        # data = response.content
    elif data := file_data.get("inline_data", None):
        name = None
        kwargs = {"data": data}
    elif name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files to "
                'Colab using the file manager ("📁 Files"in the left toolbar)'
            )
    else:
        raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")

        print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")
        return

    format = mimetypes.guess_extension(mime_type).strip(".")
    if mime_type.startswith("image/"):
        image = IPython.display.Image(**kwargs, width=256)
        IPython.display.display(image)
        print()
        return

    if mime_type.startswith("audio/"):
        if len(data) < 2**12:
            audio = IPython.display.Audio(**kwargs)
            IPython.display.display(audio)
            print()
            return

    if mime_type.startswith("video/"):
        if len(data) < 2**12:
            audio = IPython.display.Video(**kwargs, mimetype=mime_type)
            IPython.display.display(audio)
            print()
            return

    print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")

In [ ]:
for content in gais_contents:
    if role := content.get("role", None):
        print("Role:", role, "\n")

    for n, part in enumerate(content["parts"]):
        if text := part.get("text", None):
            print(text, "\n")

        elif file_data := part.get("file_data", None):
            show_file(file_data)

    print("-" * 80, "\n")

## Convert & upload files

For each file, Google AI Studio either sent:
- a Google Drive IDs
- a URL, or
- the raw bytes (`inline_data`).

The API itself onlty understands two ways of sending files:

- `inline_data` - where the bytes are placed inline in the request.
- `file_data` - where the file is uploaded to the Files API, and you pass a reference to that file.

This section goes through the `contents` from Google AI Studio, and uploads the file data, to the Files API, so Gemini can access it:

In [17]:
# @title `upload_file` function

tempfiles = pathlib.Path(f"tempfiles")
tempfiles.mkdir(parents=True, exist_ok=True)


def upload_file_data(file_data):
    mime_type = file_data["mime_type"]
    if drive_id := file_data.pop("drive_id", None):
        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        print("Uploading:", str(path))
        file_info = genai.upload_file(path=path, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if url := file_data.pop("url", None):
        response = requests.get(url)
        data = response.content
        name = url.split("/")[-1]
        hash = hashlib.sha256(data).hexdigest()
        path = tempfiles / hash
        path.write_bytes(data)
        print("Uploading:", url)
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files "
                'to Colab using the file manager ("📁 Files"in the left '
                "toolbar)"
            )
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if "inline_data" in file_data:
        return

    raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")

In [18]:
contents = copy.deepcopy(gais_contents)

for content in contents:
    for n, part in enumerate(content["parts"]):
        if file_data := part.get("file_data", None):
            upload_file_data(file_data)

Here is the coverted `Content`s:

In [19]:
contents

[]

## Call `generate_content`

In [ ]:
# Call the model and print the response.
gemini = genai.GenerativeModel(model_name=model)

response = gemini.generate_content(
    contents,
    generation_config=generation_config,
    safety_settings=safety_settings,
    stream=False,
)

In [ ]:
if generation_config.get("candidate_count", 1) == 1:
    display(Markdown(response.text))

In [ ]:
response.candidates

## Or Create a chat

In [20]:
gemini = genai.GenerativeModel(
    model_name=model,
    generation_config=generation_config,
    safety_settings=safety_settings,
)

A `ChatSession`, should always end with the model's turn.

So before creating the `chat` check whos turn was last.

If the user was last, attach all but the last content as the `history` and send the last content with `send_message` to get the model's response.

If the model was last, put the whole contents list in the history.

In [22]:
# !pip install google-generativeai

import google.generativeai as genai

# Initialize model_turn to False, assuming it's not the model's turn initially
model_turn = False

# Check if contents is not empty before accessing elements
if contents and contents[-1].get("role", None) == "user":
    model_turn = True

if model_turn:
    # If it's the model's turn, start a chat with previous history
    chat = gemini.start_chat(history=contents[:-1])

    # Send the user's message to the model
    response = chat.send_message(contents[-1])

    # If only one candidate is requested, display the response text
    if generation_config.get("candidate_count", 1) == 1:
        # !pip install markdown # Install markdown if not already installed
        from markdown import Markdown
        display(Markdown(response.text))
else:
    # If it's not the model's turn, start a new chat with the entire history
    chat = gemini.start_chat(history=contents)

In [23]:
if model_turn:
    response.candidates

After that use `send_message` to continue the conversation.

In [24]:
response = chat.send_message("Interesting, tell me more.")
display(Markdown(response.text))

Please provide me with some context!  What are you interested in learning more about? 

For example, you could tell me:

* **"Tell me more about the history of the Roman Empire."**
* **"Interesting, tell me more about the benefits of meditation."**
* **"Tell me more about the latest developments in artificial intelligence."** 

I'm here to help you explore a wide range of topics! 
